### Imputing & Adding Newly Created Variables to Testing Data

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))
print(os.listdir("../input/rossmann-store-sales"))

['exploratory-analysis', 'rossmann-store-sales']
['store.csv', 'train.csv', 'sample_submission.csv', 'test.csv']


In [3]:
types = {'CompetitionOpenSinceYear': np.dtype(int),
         'CompetitionOpenSinceMonth': np.dtype(int),
         'StateHoliday': np.dtype(str),
         'Promo2SinceWeek': np.dtype(int),
         'SchoolHoliday': np.dtype(float),
         'PromoInterval': np.dtype(str)}
training = pd.read_csv("../input/exploratory-analysis/training.csv", dtype = types)
test = pd.read_csv("../input/rossmann-store-sales/test.csv",parse_dates=[3], dtype = types)
store = pd.read_csv("../input/exploratory-analysis/storeImputed.csv")

### Filling in Missing Values

In [12]:
test[pd.isnull(test.Open)]

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
479,480,622,4,2015-09-17,NaN,1,0,0.0
1335,1336,622,3,2015-09-16,NaN,1,0,0.0
2191,2192,622,2,2015-09-15,NaN,1,0,0.0
3047,3048,622,1,2015-09-14,NaN,1,0,0.0
4759,4760,622,6,2015-09-12,NaN,0,0,0.0
5615,5616,622,5,2015-09-11,NaN,0,0,0.0
6471,6472,622,4,2015-09-10,NaN,0,0,0.0
7327,7328,622,3,2015-09-09,NaN,0,0,0.0
8183,8184,622,2,2015-09-08,NaN,0,0,0.0
9039,9040,622,1,2015-09-07,NaN,0,0,0.0


In [22]:
test['Open'].fillna(test['Open'].mode()[0], inplace=True)
print(test.isnull().sum())

Id               0
Store            0
DayOfWeek        0
Date             0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64


Since all of the missing Open values in testing data aren't associated with a state holiday, the missing values are filled in with the mode. 

In [26]:
# merging store & test data
testing = pd.merge(test, store, on='Store')
print(testing.isnull().sum())

Id                           0
Store                        0
DayOfWeek                    0
Date                         0
Open                         0
Promo                        0
StateHoliday                 0
SchoolHoliday                0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoInterval                0
dtype: int64


#### Adding Newly Created Variables

In [27]:
# dates breakdown
testing['Date'] = pd.to_datetime(testing['Date'])
testing['Day_of_Week'] = testing['Date'].dt.day_name()

testing['Date'] = pd.to_datetime(testing['Date'])
testing['Month'] = testing['Date'].dt.strftime('%b')

# competition dates
testing["CompOpenDate"] = testing["CompetitionOpenSinceYear"].astype(str) + testing["CompetitionOpenSinceMonth"].astype(str) + '1'
testing['CompOpenDate'] = pd.to_datetime(testing['CompOpenDate'])

def check_Comp(row):
    if row["Date"] >= row["CompOpenDate"]:
        return "1"
    else:
        return "0"
    
def num_days_comp(row):
    return row['Date'] - row['CompOpenDate']

testing = testing.assign(daysSinceNewComp=testing.apply(num_days_comp, axis=1))
testing['daysSinceNewComp'] = testing['daysSinceNewComp'].dt.days
testing = testing.assign(isCompOpen=testing.apply(check_Comp, axis=1))

# Competition Distance
def comp_dist(row):
    if row["CompetitionDistance"] <= 710: # min to first quartile
        return "bin1"
    elif row["CompetitionDistance"] > 710 and row["CompetitionDistance"] <= 2325 : # first quartile to second quartile
        return "bin2"
    elif row["CompetitionDistance"] > 2325 and row["CompetitionDistance"] <= 6880 : # second quartile to third quartile
        return "bin3"
    elif row["CompetitionDistance"] > 6880 and row["CompetitionDistance"] <= 75860 : # third quartile to max
        return "bin4"

testing = testing.assign(compDistanceBin=testing.apply(comp_dist, axis=1))



In [28]:
training.shape

(844338, 24)

In [29]:
testing.shape

(41088, 23)

In [30]:
testing.to_csv("testing.csv", index=False)

As expected, the # of columns match, except by one, since the testing data won't include # of customers.